# Business Inteligence Sheet 6

### Juan David Camacho
### Hernán Russi
### Juan Esteban Daza  


1. Write a function agglomerativeClustering(D, dist, k=1) that implements the agglomerative clustering algorithm with k clusters. Here, dist is a function so that
dist(D, Ci, Cj) returns the distance between clusters Ci, Cj (lists point indices),
and D is the original dataset. The clustering algorithm should return a list C of same
length as D with the cluster ids (numbers from 0 to k-1) for each point.
Implement the five distance metrics using the function names singleLink, completeLink,
groupAverage, meanDistance, and ward.

In [101]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import statistics

In [102]:

# Single Link
def singleLink(D, Ci, Cj):
    distances = []
    for ci in Ci:
        x = D[ci, :]
        for cj in Cj:
            y = D[cj, :]
            distances.append(np.linalg.norm(x - y))

    return np.amin(distances)

# Complete Link
def completeLink(D, Ci, Cj):
    distances = []
    for ci in Ci:
        x = D[ci, :]
        for cj in Cj:
            y = D[cj, :]
            distances.append(np.linalg.norm(x - y))

    return np.amax(distances)

# Group Avarage
def groupAverage(D, Ci, Cj):
    distances = []
    for ci in Ci:
        x = D[ci, :]
        for cj in Cj:
            y = D[cj, :]
            distances.append(np.linalg.norm(x - y))

    return sum(distances)/len(Ci)*len(Cj)

# Mean Distance
def meanDistance(D, Ci, Cj):
    items_Ci = []
    items_Cj = []
    for ci in Ci:
        items_Ci.append(D[ci, :])
    for cj in Cj:
        items_Cj.append(D[cj, :])

    mean_ci = sum(items_Ci)/len(items_Ci)
    mean_cj = sum(items_Cj)/len(items_Cj)
    return np.linalg.norm(mean_ci - mean_cj)

#Ward Distance
def wardDistance(D, Ci, Cj):
    items_Ci = []
    items_Cj = []
    for ci in Ci:
        items_Ci.append(D[ci, :])
    for cj in Cj:
        items_Cj.append(D[cj, :])

    mean_ci = sum(items_Ci)/len(items_Ci)
    mean_cj = sum(items_Cj)/len(items_Cj)
    return ((len(items_Ci)*len(items_Cj))/(len(items_Ci) + len(items_Cj))) * np.linalg.norm(mean_ci - mean_cj)**2


In [103]:
#Our function from sheet 5 XD
def getDistances(A):
    rows, columns = A.shape
    matrix_w = np.zeros((rows, rows))
        
    for row in range(0, rows):
        first_sel_row = A[row,:]
        for second_row in range(0, rows):
            second_sel_row = A[second_row,:]
            matrix_w[row, second_row] = np.linalg.norm(first_sel_row - second_sel_row)

    return matrix_w

def agglomerativeClustering(D, dist, k=1):
    C = list(range(0, len(D)))
    distance_matrix = getDistances(D)
    while True:
        distance_matrix[distance_matrix == 0] = np.inf

        #c = np.where(distance_matrix == np.min(distance_matrix[np.nonzero(distance_matrix)]))
        #np.triu(distance_matrix)
        c = np.unravel_index(distance_matrix.argmin(), distance_matrix.shape)
        

        greater = np.amax(c)
        lesser = np.amin(c)

        c = (lesser, greater)       
        #print('RA', c)
        greater_cluster = 0
        first_choosing = c[0]
        for item2 in range(1, len(c)):
            if first_choosing != c[item2]:
                greater_cluster = c[item2]

        for idx, value in enumerate(C):

            if(value == greater_cluster):

                C[idx] = c[0]

        distance_matrix = np.zeros((len(C), len(C)))

        for row in set(C):
            ci = []
            for idx, finding in enumerate(C):
                if(row == finding):
                    ci.append(idx)
            for row2 in set(C):
                cj = []
                for idx, finding in enumerate(C):
                    if(row2 == finding):
                        cj.append(idx)

                if(row != row2):
                    distance_matrix[row, row2] = dist(D, ci, cj)
                    distance_matrix[row2, row] = dist(D, ci, cj)
        #print(distance_matrix[0,4])
        if(len(set(C)) == k):
            break

    return(C)


2. Write a function agglomerativeClusteringLW(D, dist, k=1) that does the same clustering but uses the Lance-Williams formula. This time, dist should be a string in
{single, complete, groupavg, meandist, ward}, and your algorithm should automatically
configure the correct params for αi
, αj , β, γ.


In [104]:


def get_distance(D, dist, Ci, Cj):
    if dist == 'single':
        return singleLink(D, Ci, Cj)
    elif dist == 'complete':
        return completeLink(D, Ci, Cj)
    elif dist == 'groupavg':
        return groupAverage(D, Ci, Cj)
    elif dist == 'meandist':
        return meanDistance(D, Ci, Cj)
    elif dist == 'ward':
        return wardDistance(D, Ci, Cj)

def agglomerativeClusteringLW(D, dist, k=1):
    C = list(range(0, len(D)))
    distance_matrix = getDistances(D)

    while True:
        distance_matrix[distance_matrix == 0] = np.inf

        #c = np.where(distance_matrix == np.min(distance_matrix[np.nonzero(distance_matrix)]))
        c = np.unravel_index(distance_matrix.argmin(), distance_matrix.shape)
        #print('LH', c)
        greater_cluster = 0
        ci = []
        cj = []
        cr = []

        for idx, value in enumerate(C):
            if(value == c[0]):
                ci.append(idx)

        for idx, value in enumerate(C):
            if(value == c[1]):
                cj.append(idx)

        #print(C)
        #print(ci, cj)
        distance_matrix[c[0],:] = 0
        distance_matrix[:,c[0]] = 0
        distance_matrix[c[1],:] = 0
        distance_matrix[:,c[1]] = 0
        for row in set(C):
            cr = []
            if(c[0] != row and c[1] != row):
                for idx, finding in enumerate(C):
                    if(row == finding):
                        cr.append(idx)
                if dist == 'single':
                    alpha_i = 1/2
                    alpha_j = 1/2
                    beta = 0
                    gamma = -1/2
                elif dist == 'complete':
                    alpha_i = 1/2
                    alpha_j = 1/2
                    beta = 0
                    gamma = 1/2
                elif dist == 'groupavg':
                    alpha_i = len(ci)/(len(ci)+len(cj))
                    alpha_j = len(cj)/(len(ci)+len(cj))
                    beta = 0
                    gamma = 0
                    #print(len(ci), len(cj))
                elif dist == 'meandist':
                    alpha_i = len(ci)/(len(ci)+len(cj))
                    alpha_j = len(cj)/(len(ci)+len(cj))
                    beta = (-len(ci)*len(cj))/((len(ci)+len(cj))**2)
                    gamma = 0
                elif dist == 'ward':
                    alpha_i = (len(ci) + len(cr))/(len(ci)+len(cj) + len(cr))
                    alpha_j = (len(cj) + len(cr))/(len(ci)+len(cj) + len(cr))
                    beta = -len(cr)/(len(ci)+len(cj)+len(cr))
                    gamma = 0
                
            
                calculus = alpha_i * get_distance(D, dist, ci, cr) + alpha_j * get_distance(D, dist, cj, cr) + beta * get_distance(D, dist, ci, cj) + (gamma * abs(get_distance(D, dist, ci, cr) - get_distance(D, dist, cj, cr)))
                distance_matrix[c[0], row] = calculus
                distance_matrix[row, c[0]] = calculus
        #print(distance_matrix)
        first_choosing = c[0]
        for item2 in range(1, len(c)):
            if first_choosing != c[item2]:
                greater_cluster = c[item2]

        for idx, value in enumerate(C):

            if(value == greater_cluster):

                C[idx] = c[0]
       #print(distance_matrix[0,4])
        if(len(set(C)) == k):
            break
        
    return(C)


## Exercise 2 (3 points)
1. Compare the runtimes of agglomerativeClustering and agglomerativeClusteringLW
on the mall customer dataset once using single link and once using group average for
k = 5. Report the runtimes and interpret them.

In [107]:
import timeit

mall = pd.get_dummies(pd.read_csv("Mall_Customers.csv")).values

start = timeit.default_timer()
agglomerativeClustering(mall, singleLink, 5)
stop = timeit.default_timer()
runtime_ag_single = stop - start

start = timeit.default_timer()
agglomerativeClusteringLW(mall, 'single', 5)
stop = timeit.default_timer()
runtime_aglw_single = stop - start

start = timeit.default_timer()
agglomerativeClustering(mall, groupAverage, 5)
stop = timeit.default_timer()
runtime_ag_ga = stop - start

start = timeit.default_timer()
agglomerativeClusteringLW(mall, 'groupavg', 5)
stop = timeit.default_timer()
runtime_aglw_ga = stop - start

print('Runtime Aglo Clustering single: ', runtime_ag_single)
print('Runtime Aglo Clustering LW single: ', runtime_aglw_single)
print('Runtime Aglo Clustering Group Avg: ', runtime_ag_ga)
print('Runtime Aglo Clustering LW Group Avg: ', runtime_aglw_ga)




RA [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 123, 124, 123, 0, 123, 128, 123, 128, 123, 124, 123, 128, 123, 128, 123, 128, 123, 128, 123, 124, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 128, 123, 198, 123]
LH [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

The diference between runtimes is way too big to even consider using the method that is not Lance Williams. The difference can be noted when we compute the distance matrix in the method without Lance Williams, since we re-compute all the matrix, while in the Lance Williams method we only re-compute the corresponding row/column for the merged cluster. 